In [2]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    print(f"[GPU setup] {len(gpus)} Physical GPU(s) with memory growth enabled.")
else:
    print("[GPU setup] No GPU found, using CPU.")
import os
import warnings
warnings.filterwarnings('ignore')


# Core libraries
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input  # Changed to ResNet
from sklearn.model_selection import train_test_split
from tqdm import tqdm

[GPU setup] 1 Physical GPU(s) with memory growth enabled.


# Build Pretrained ResNet50 Model

In [3]:
def build_resnet(dropout_rate=0.5):  
    base = ResNet50(weights='imagenet', include_top=False,
                   input_shape=(224, 224, 3))  
    for layer in base.layers:
        layer.trainable = False

    x = Flatten()(base.output)
    x = Dense(256, activation='relu')(x)
    x = Dropout(dropout_rate)(x)
    out = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=base.input, outputs=out)
    return model

# Prepare Dataset & Generators 

In [4]:
# Paths
import os
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_dir = "/kaggle/input/cifake-real-and-ai-generated-synthetic-images/train"
val_dir  = "/kaggle/working/validation"

# Create validation split
for cls in ['REAL', 'FAKE']:
    os.makedirs(os.path.join(val_dir, cls), exist_ok=True)
    files = os.listdir(os.path.join(data_dir, cls))
    _, val_files = train_test_split(files, test_size=0.2, random_state=42)
    for f in val_files:
        src = os.path.join(data_dir, cls, f)
        dst = os.path.join(val_dir, cls, f)
        shutil.copyfile(src, dst)

# Generators (using ResNet's preprocess_input)
train_gen = ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=True
)

val_gen = ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

test_gen = ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_directory(
    "/kaggle/input/cifake-real-and-ai-generated-synthetic-images/test",
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

Found 100000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.


# Adversarial Attack Functions 

In [5]:
@tf.function
def fgsm_attack(model, images, labels, epsilon=0.05):
    with tf.GradientTape() as tape:
        tape.watch(images)
        preds = model(images)
        loss = tf.keras.losses.BinaryCrossentropy()(labels, preds)
    grad = tape.gradient(loss, images)
    adv = images + epsilon * tf.sign(grad)
    return tf.clip_by_value(adv, -1.0, 1.0)

@tf.function
def pgd_attack(x, y, model, loss_fn,
               epsilon=0.03, alpha=0.007, iters=10):
    x_adv = tf.identity(x)
    for _ in range(iters):
        with tf.GradientTape() as tape:
            tape.watch(x_adv)
            pred = model(x_adv, training=False)
            loss = loss_fn(y, pred)
        grad = tape.gradient(loss, x_adv)
        x_adv = x_adv + alpha * tf.sign(grad)
        x_adv = tf.clip_by_value(x_adv, x - epsilon, x + epsilon)
        x_adv = tf.clip_by_value(x_adv, -1.0, 1.0)
    return x_adv

# Train & Save Baseline ResNet Model

In [6]:
baseline_weights = "/kaggle/working/resnet_baseline.weights.h5"  
baseline_model_path = "/kaggle/working/resnet_baseline_model.h5"  

In [7]:
if os.path.exists(baseline_model_path):
    baseline_model = load_model(baseline_model_path)
    print("Loaded existing baseline model.")
else:
    baseline_model = build_resnet() 
    baseline_model.compile(optimizer='adam',
                         loss='binary_crossentropy',
                         metrics=['accuracy'])
    baseline_model.fit(
        train_gen, epochs=3, validation_data=val_gen)
    baseline_model.save_weights(baseline_weights)
    baseline_model.save(baseline_model_path)
    print("ResNet baseline model trained and saved.")  

I0000 00:00:1745093400.373898      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/3


I0000 00:00:1745093415.024030     113 service.cc:148] XLA service 0x79e6d004d2c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745093415.024832     113 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1745093416.453428     113 cuda_dnn.cc:529] Loaded cuDNN version 90300


   2/3125 ━━━━━━━━━━━━━━━━━━━━ 2:54 56ms/step - accuracy: 0.6094 - loss: 20.2341  

I0000 00:00:1745093420.704563     113 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3125/3125 ━━━━━━━━━━━━━━━━━━━━ 461s 142ms/step - accuracy: 0.8750 - loss: 0.8578 - val_accuracy: 0.9395 - val_loss: 0.1617
Epoch 2/3
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 255s 81ms/step - accuracy: 0.9258 - loss: 0.1933 - val_accuracy: 0.9568 - val_loss: 0.1172
Epoch 3/3
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 215s 69ms/step - accuracy: 0.9372 - loss: 0.1630 - val_accuracy: 0.9655 - val_loss: 0.0860
ResNet baseline model trained and saved.


# Baseline Evaluation 

In [8]:
baseline_model.compile(optimizer='adam',
                     loss='binary_crossentropy',
                     metrics=['accuracy'])

In [9]:
import numpy as np
import tensorflow as tf

num_batches = 20
clean_accs = []
fgsm_accs  = []
pgd_accs   = []
loss_fn = tf.keras.losses.BinaryCrossentropy()
test_gen.reset()

for i in range(num_batches):
    x_batch, y_batch = next(test_gen)
    
    # Clean accuracy
    preds_clean = baseline_model(x_batch, training=False)
    acc_clean = tf.reduce_mean(tf.cast(tf.equal(tf.round(preds_clean), y_batch), tf.float32)).numpy()
    clean_accs.append(acc_clean)

    # FGSM
    x_adv_fgsm = fgsm_attack(baseline_model, tf.convert_to_tensor(x_batch), 
                            tf.convert_to_tensor(y_batch)).numpy()
    preds_fgsm = baseline_model(x_adv_fgsm, training=False)
    acc_fgsm = tf.reduce_mean(tf.cast(tf.equal(tf.round(preds_fgsm), y_batch), tf.float32)).numpy()
    fgsm_accs.append(acc_fgsm)

    # PGD
    x_adv_pgd = pgd_attack(tf.convert_to_tensor(x_batch), y_batch, 
                          baseline_model, loss_fn).numpy()
    preds_pgd = baseline_model(x_adv_pgd, training=False)
    acc_pgd = tf.reduce_mean(tf.cast(tf.equal(tf.round(preds_pgd), y_batch), tf.float32)).numpy()
    pgd_accs.append(acc_pgd)

print(f"Clean Test Accuracy (avg over {num_batches} batches): {np.mean(clean_accs):.4f}")
print(f"FGSM Accuracy      (avg over {num_batches} batches): {np.mean(fgsm_accs):.4f}")
print(f"PGD Accuracy       (avg over {num_batches} batches): {np.mean(pgd_accs):.4f}")

Clean Test Accuracy (avg over 20 batches): 0.9250
FGSM Accuracy      (avg over 20 batches): 0.1328
PGD Accuracy       (avg over 20 batches): 0.1219


# Generate & Save Perturbed Training Data 

In [19]:
pert_dir = "/kaggle/working/train_perturbed"
for cls in ['REAL','FAKE']:
    os.makedirs(os.path.join(pert_dir,cls), exist_ok=True)

# Reload baseline
baseline_model = build_resnet()  # Changed to ResNet
baseline_model.load_weights(baseline_weights)

no_shuffle = ImageDataGenerator(preprocessing_function=preprocess_input)
no_shuffle = no_shuffle.flow_from_directory(
    data_dir, target_size=(224,224), batch_size=32,
    class_mode='binary', shuffle=False)

loss_fn = tf.keras.losses.BinaryCrossentropy()
num_batches = len(no_shuffle)

for i in tqdm(range(num_batches)):
    xb, yb = next(no_shuffle)
    idx = no_shuffle.index_array[i*32:(i+1)*32]
    paths = [no_shuffle.filepaths[k] for k in idx]
    half = xb.shape[0]//2

    x_f = fgsm_attack(baseline_model,
                     tf.convert_to_tensor(xb[:half]),
                     tf.convert_to_tensor(yb[:half])).numpy()
    x_p = pgd_attack(tf.convert_to_tensor(xb[half:]),
                     yb[half:], baseline_model, loss_fn).numpy()
    x_comb = np.concatenate([x_f, x_p], axis=0)

    for j, pth in enumerate(paths):
        cls_name = 'REAL' if 'REAL' in pth else 'FAKE'
        fn = os.path.basename(pth).replace('.jpg', f'_adv_{i}_{j}.jpg')
        savep = os.path.join(pert_dir, cls_name, fn)
        tf.keras.preprocessing.image.save_img(savep, x_comb[j])

Found 100000 images belonging to 2 classes.


100%|██████████| 3125/3125 [47:36<00:00,  1.09it/s] 


# Adversarial Training & Saving 

In [22]:
pert_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
pert_gen = pert_gen.flow_from_directory(
    pert_dir, target_size=(224,224), batch_size=32,
    class_mode='binary', shuffle=True)

adv_model = build_resnet()  
adv_model.compile(optimizer='adam', loss='binary_crossentropy',
                metrics=['accuracy'])
adv_model.fit(pert_gen, epochs=3, validation_data=val_gen)
adv_model.save('/kaggle/working/resnet_adv_model.h5')  



Found 100000 images belonging to 2 classes.
Epoch 1/3
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 199s 60ms/step - accuracy: 0.7165 - loss: 1.3159 - val_accuracy: 0.6726 - val_loss: 0.7969
Epoch 2/3
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 183s 58ms/step - accuracy: 0.7776 - loss: 0.4500 - val_accuracy: 0.6720 - val_loss: 0.8774
Epoch 3/3
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 184s 59ms/step - accuracy: 0.7998 - loss: 0.4136 - val_accuracy: 0.6399 - val_loss: 1.3010


# Robustness Evaluation 

In [26]:
# FGSM evaluation
x_t, y_t = next(test_gen)
x_t_tensor = tf.convert_to_tensor(x_t)
y_t_tensor = tf.convert_to_tensor(y_t)
x_adv_fgsm = fgsm_attack(adv_model, x_t_tensor, y_t_tensor, epsilon=0.05).numpy()
fgsm_loss, fgsm_acc = adv_model.evaluate(x_adv_fgsm, y_t, verbose=0)
print(f"FGSM Accuracy on Robust Model: {fgsm_acc:.4f}")

# PGD evaluation
x_p, y_p = next(test_gen) 
x_p_tensor = tf.convert_to_tensor(x_p)
y_p_tensor = tf.convert_to_tensor(y_p)

x_adv_pgd = pgd_attack(
    x_p_tensor,
    y_p_tensor,
    adv_model,
    loss_fn,
    epsilon=0.03,
    alpha=0.007,
    iters=10
).numpy()

pgd_loss, pgd_acc = adv_model.evaluate(x_adv_pgd, y_p, verbose=0)
print(f"PGD Accuracy on Robust Model: {pgd_acc:.4f}")

FGSM Accuracy on Robust Model: 0.3750
PGD Accuracy on Robust Model: 0.2812
